# Read Table

In [13]:
import os
import json
import boto3
import pprint
import requests
import numpy as np
import pandas as pd
import datetime as dt
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

print("imported modules.")


imported modules.


## Spark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from reddit.reddit_streaming import *
import datetime as dt
import pprint
pp = pprint.PrettyPrinter(indent = 1)
%load_ext sparksql_magic

creds, config = read_files()
subreddit_list = config["subreddit"]
kafka_host = config["kafka_host"]
spark_host = config["spark_host"]
aws_client = creds["aws-client"]
aws_secret = creds["aws-secret"]                    

for subreddit in subreddit_list:
    spark = SparkSession.builder.appName("reddit_" + subreddit + "_read_data") \
                        .master("spark://{}:7077".format(spark_host)) \
                        .config("spark.scheduler.mode", "FAIR") \
                        .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
                        .config("spark.executor.memory", "1024m") \
                        .config("spark.executor.cores", "2") \
                        .config("spark.streaming.concurrentJobs", "4") \
                        .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
                        .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
                        .config("spark.sql.debug.maxToStringFields", 1000) \
                        .config("spark.eventLog.enabled", "true") \
                        .config("spark.eventLog.dir", "file:///opt/workspace/events") \
                        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-core_2.12:1.2.1") \
                        .config("spark.hadoop.fs.s3a.access.key", aws_client) \
                        .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
                        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                        .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
                        .enableHiveSupport() \
                        .getOrCreate()

    print("created spark successfully.")

    df = spark.read.format("delta").option("header", True).load("s3a://reddit-stevenhurwitt/" + subreddit + "_clean/")
    
    df.createOrReplaceTempView("reddit_{}".format(subreddit))
    print("created table reddit_{}.".format(subreddit))

The sparksql_magic extension is already loaded. To reload it, use:
  %reload_ext sparksql_magic
created spark successfully.


created table reddit_technology.
created spark successfully.


created table reddit_ProgrammerHumor.
created spark successfully.


created table reddit_news.
created spark successfully.


created table reddit_worldnews.


## Athena

In [14]:
pp = pprint.PrettyPrinter(indent = 1)

subreddit = "technology" 
athena = boto3.client('athena')
df = athena.start_query_execution(
         QueryString = "select * from reddit.{}".format(subreddit),
         ResultConfiguration = {
             'OutputLocation': "s3://reddit-stevenhurwitt/_athena_results"
         })

# pp.pprint(df)

query_execution_id = df["QueryExecutionId"]
request_id = df["ResponseMetadata"]["RequestId"]
print("query execution id : {}".format(query_execution_id))
print("request id: {}".format(request_id))

query execution id : 7dbbeb09-dde2-44f3-9bbd-3aadd73cdee9
request id: 9b32cc64-9524-4ca1-9342-37be1042d6aa


In [24]:
response = athena.get_query_results(QueryExecutionId=query_execution_id, MaxResults=1000)
df = response["ResultSet"]["Rows"][0]["Data"]
# df = pd.DataFrame.from_records(df)
df

[{'VarCharValue': 'approved_at_utc'},
 {'VarCharValue': 'subreddit'},
 {'VarCharValue': 'selftext'},
 {'VarCharValue': 'author_fullname'},
 {'VarCharValue': 'saved'},
 {'VarCharValue': 'mod_reason_title'},
 {'VarCharValue': 'gilded'},
 {'VarCharValue': 'clicked'},
 {'VarCharValue': 'title'},
 {'VarCharValue': 'subreddit_name_prefixed'},
 {'VarCharValue': 'hidden'},
 {'VarCharValue': 'pwls'},
 {'VarCharValue': 'link_flair_css_class'},
 {'VarCharValue': 'downs'},
 {'VarCharValue': 'thumbnail_height'},
 {'VarCharValue': 'top_awarded_type'},
 {'VarCharValue': 'hide_score'},
 {'VarCharValue': 'name'},
 {'VarCharValue': 'quarantine'},
 {'VarCharValue': 'link_flair_text_color'},
 {'VarCharValue': 'upvote_ratio'},
 {'VarCharValue': 'author_flair_background_color'},
 {'VarCharValue': 'ups'},
 {'VarCharValue': 'total_awards_received'},
 {'VarCharValue': 'thumbnail_width'},
 {'VarCharValue': 'author_flair_template_id'},
 {'VarCharValue': 'is_original_content'},
 {'VarCharValue': 'secure_media'},


In [29]:
pp.pprint(df)

[{'VarCharValue': 'approved_at_utc'},
 {'VarCharValue': 'subreddit'},
 {'VarCharValue': 'selftext'},
 {'VarCharValue': 'author_fullname'},
 {'VarCharValue': 'saved'},
 {'VarCharValue': 'mod_reason_title'},
 {'VarCharValue': 'gilded'},
 {'VarCharValue': 'clicked'},
 {'VarCharValue': 'title'},
 {'VarCharValue': 'subreddit_name_prefixed'},
 {'VarCharValue': 'hidden'},
 {'VarCharValue': 'pwls'},
 {'VarCharValue': 'link_flair_css_class'},
 {'VarCharValue': 'downs'},
 {'VarCharValue': 'thumbnail_height'},
 {'VarCharValue': 'top_awarded_type'},
 {'VarCharValue': 'hide_score'},
 {'VarCharValue': 'name'},
 {'VarCharValue': 'quarantine'},
 {'VarCharValue': 'link_flair_text_color'},
 {'VarCharValue': 'upvote_ratio'},
 {'VarCharValue': 'author_flair_background_color'},
 {'VarCharValue': 'ups'},
 {'VarCharValue': 'total_awards_received'},
 {'VarCharValue': 'thumbnail_width'},
 {'VarCharValue': 'author_flair_template_id'},
 {'VarCharValue': 'is_original_content'},
 {'VarCharValue': 'secure_media'},


In [28]:
pd.DataFrame.from_dict(df, orient = "columns")

,VarCharValue
0,approved_at_utc
1,subreddit
2,selftext
3,author_fullname
4,saved
...,...
98,is_video
99,date
100,year
101,month


## technology

In [5]:
%%sparksql

select post_date, count(*) from reddit_technology 
group by post_date 
order by post_date asc

post_date,count(1)
2022-05-11,3
2022-05-12,32
2022-05-13,27
2022-05-16,15
2022-06-14,43
2022-06-15,42
2022-06-16,41
2022-06-17,22
2022-06-18,15
2022-06-19,37


In [4]:
%%sparksql

select * from reddit_technology order by created_utc asc limit 10

approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,post_hint,content_categories,is_self,subreddit_type,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,post_date
null,technology,,t2_8pgkdmnc,False,null,0,False,Tech is hitting the brakes on hiring even as other industries keep adding jobs,r/technology,False,6,general,0,93,null,True,t3_unm15z,False,dark,1.0,null,3,0,140,null,False,null,False,False,null,Business,False,3,null,False,False,https://a.thumbs.redditmedia.com/0YC2GoTdt-YfMeBaTlYL2FNrrNVCUUSfaPFXbgnN4Q4.jpg,False,null,link,null,False,public,2022-05-11 22:38:01.162752,text,6,null,null,text,cnbc.com,False,null,null,null,null,https://www.cnbc.com/amp/2022/05/11/tech-layoffs-hiring-slowdown-stand-out-in-red-hot-job-market.html,null,False,False,True,False,False,False,49cac61c-a816-11e9-be34-0ebbab5890a0,True,False,False,null,False,null,null,null,t5_2qh16,False,null,null,null,,unm15z,True,null,saurin212,null,1,True,all_ads,False,False,null,/r/technology/comments/unm15z/tech_is_hitting_the_brakes_on_hiring_even_as/,all_ads,False,https://www.cnbc.com/amp/2022/05/11/tech-layoffs-hiring-slowdown-stand-out-in-red-hot-job-market.html,11961363,2022-05-11 22:38:01.162752,0,null,False,2022-05-11
null,technology,,t2_50rzo,False,null,0,False,"Appeals court lifts stay, allows TX social media law to go into effect",r/technology,False,6,general,0,70,null,True,t3_unm8vy,False,dark,1.0,null,3,0,140,null,False,null,False,False,null,Politics,False,3,null,False,False,https://b.thumbs.redditmedia.com/nQHl8WXhGBisDJXlmDGVXpNRldbpWMH1xldfflbxWAc.jpg,False,null,link,null,False,public,2022-05-11 22:49:12.251392,text,6,null,null,text,protocol.com,False,null,null,null,null,https://www.protocol.com/bulletins/texas-bias-law-proceeds,null,False,True,True,False,False,False,70f05306-a816-11e9-b261-0e41843d240e,True,False,False,null,False,null,null,null,t5_2qh16,False,null,null,null,,unm8vy,True,null,macnbc,null,0,True,all_ads,False,False,null,/r/technology/comments/unm8vy/appeals_court_lifts_stay_allows_tx_social_media/,all_ads,False,https://www.protocol.com/bulletins/texas-bias-law-proceeds,11961400,2022-05-11 22:49:12.251392,0,null,False,2022-05-11
null,technology,,t2_5w9r3,False,null,0,False,Court lets Texas restrictions on social platform content moderation take effect,r/technology,False,6,general,0,73,null,True,t3_unmsdt,False,dark,1.0,null,2,0,140,null,False,null,False,False,null,Social Media,False,2,null,False,True,https://b.thumbs.redditmedia.com/ncGR-bgoKIUbuE-gMQY2lAODE5fyDOww-ZF78xEBk0s.jpg,False,null,link,null,False,public,2022-05-11 23:18:17.081856,text,6,null,null,text,theverge.com,False,null,null,null,null,https://www.theverge.com/2022/5/11/23067002/texas-netchoice-paxton-hb20-social-media-law-fifth-circuit-appeals-court-grant-stay-ruling,null,False,False,True,False,False,False,7d4d8376-a8

## news

In [12]:
%%sparksql

select post_date, count(*) from reddit_news
group by post_date 
order by post_date asc

post_date,count(1)
2022-06-17,14
2022-06-18,43
2022-06-19,36
2022-06-20,24
2022-06-21,24
2022-06-22,34
2022-06-23,29
2022-06-24,36
2022-06-25,28


In [6]:
%%sparksql

select * from reddit_news order by created_utc asc limit 10

approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,post_hint,content_categories,is_self,subreddit_type,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,post_date
null,news,,t2_bs1la4xg,False,null,0,False,"Mortgage rates hit 5.78%, the biggest weekly jump since 1987",r/news,False,6,null,0,null,null,True,t3_ved5yu,False,dark,0.9599999785423279,null,54,0,null,null,False,null,False,False,null,null,False,54,null,False,False,,False,null,null,null,False,public,2022-06-17 13:25:27.229440,text,6,null,null,text,cnn.com,False,null,null,null,null,https://www.cnn.com/2022/06/16/homes/mortgage-rates-june-16/index.html,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh3l,False,null,null,null,,ved5yu,True,null,Dizzy-Noise-583,null,16,True,all_ads,False,False,null,/r/news/comments/ved5yu/mortgage_rates_hit_578_the_biggest_weekly_jump/,all_ads,False,https://www.cnn.com/2022/06/16/homes/mortgage-rates-june-16/index.html,24820150,2022-06-17 13:25:27.229440,0,null,False,2022-06-17
null,news,,t2_tlqk8,False,null,0,False,"West Memphis police investigating Friday shooting incident, 2-year-old dies from injuries",r/news,False,6,null,0,null,null,True,t3_vee3iw,False,dark,1.0,null,4,0,null,null,False,null,False,False,null,null,False,4,null,False,False,,False,null,null,null,False,public,2022-06-17 14:10:11.584000,text,6,null,null,text,katv.com,False,null,null,null,null,https://katv.com/news/local/west-memphis-police-investigating-friday-shooting-incident-2-year-old-dies-from-injury-baptist-memorial-hospital-crittenden-crittenden-county-crime-stoppers-crime-scene-interview-witnesses-arkansas-news,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh3l,False,null,null,null,,vee3iw,True,null,Dillatrack,null,0,True,all_ads,False,False,null,/r/news/comments/vee3iw/west_memphis_police_investigating_friday_shooting/,all_ads,False,https://katv.com/news/local/west-memphis-police-investigating-friday-shooting-incident-2-year-old-dies-from-injury-baptist-memorial-hospital-crittenden-crittenden-county-crime-stoppers-crime-scene-interview-witnesses-arkansas-news,24820176,2022-06-17 14:10:11.584000,0,null,False,2022-06-17
null,news,,t2_3qwsyobo,False,null,0,False,‘Unacceptable’: Russian warship accused of violating Danish waters,r/news,False,6,null,0,null,null,True,t3_veebzd,False,dark,1.0,null,3,0,null,null,False,null,False,False,null,null,False,3,null,False,False,,False,null,null,null,False,public,2022-06-17 14:21:22.672640,text,6,null,null,text,theguardian.com,False,null,null,null,null,https://www.theguardian.com/world/2022/jun/17/unacceptable-russian-warship-accused-of-violating-danish-waters,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh3l,False,null,null,null,,veebzd,True,null,B

## worldnews

In [13]:
%%sparksql

select post_date, count(*) from reddit_worldnews
group by post_date
order by post_date asc

post_date,count(1)
2022-06-17,32
2022-06-18,32
2022-06-19,106
2022-06-20,92
2022-06-21,80
2022-06-22,100
2022-06-23,102
2022-06-24,45
2022-06-25,75


In [8]:
%%sparksql

select * from reddit_worldnews order by created_utc asc limit 10

approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,post_hint,content_categories,is_self,subreddit_type,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,post_date
null,worldnews,,t2_yl06t,False,null,0,False,European Commission recommends granting Ukraine EU candidate status,r/worldnews,False,6,russia,0,null,null,True,t3_vednyc,False,null,1.0,null,9,0,null,null,False,null,False,False,null,Russia/Ukraine,False,9,null,False,True,,False,null,null,null,False,public,2022-06-17 13:52:17.842176,richtext,6,null,null,text,ukrinform.net,False,null,null,null,null,https://www.ukrinform.net/rubric-polytics/3508964-european-commission-recommends-granting-ukraine-eu-candidate-status.html,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh13,False,null,null,null,null,vednyc,True,null,Ekaton,null,2,True,all_ads,False,False,null,/r/worldnews/comments/vednyc/european_commission_recommends_granting_ukraine/,all_ads,False,https://www.ukrinform.net/rubric-polytics/3508964-european-commission-recommends-granting-ukraine-eu-candidate-status.html,29034211,2022-06-17 13:52:17.842176,0,null,False,2022-06-17
null,worldnews,,t2_bmncpj69,False,null,0,False,China blocks joint India-US proposal to blacklist Pakistani terrorist Abdul Rehman Makki under UNSC sanctions,r/worldnews,False,6,null,0,null,null,True,t3_vedzuc,False,dark,1.0,null,4,0,null,null,False,null,False,False,null,null,False,4,null,False,False,,False,null,null,null,False,public,2022-06-17 14:07:57.366272,text,6,null,null,text,tribuneindia.com,False,null,null,null,null,https://www.tribuneindia.com/news/nation/china-blocks-joint-india-us-proposal-to-blacklist-pakistani-terrorist-abdul-rehman-makki-under-unsc-sanctions-404681,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh13,False,null,null,null,,vedzuc,True,null,Winterisbucky,null,0,True,all_ads,False,False,null,/r/worldnews/comments/vedzuc/china_blocks_joint_indiaus_proposal_to_blacklist/,all_ads,False,https://www.tribuneindia.com/news/nation/china-blocks-joint-india-us-proposal-to-blacklist-pakistani-terrorist-abdul-rehman-makki-under-unsc-sanctions-404681,29034225,2022-06-17 14:07:57.366272,0,null,False,2022-06-17
null,worldnews,,t2_3z7n2wc1,False,null,0,False,UK: 50 new monkeypox cases in biggest epidemic beyond Africa,r/worldnews,False,6,null,0,null,null,True,t3_vee2pe,False,dark,1.0,null,3,0,null,null,False,null,False,False,null,null,False,3,null,False,True,,False,null,null,null,False,public,2022-06-17 14:10:11.584000,text,6,null,null,text,thehill.com,False,null,null,null,null,https://thehill.com/homenews/ap/ap-health/uk-50-new-monkeypox-cases-in-biggest-epidemic-beyond-africa/,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2qh13,False,null,null,null,,vee2pe

## ProgrammerHumor

In [14]:
%%sparksql

select post_date, count(*) from reddit_ProgrammerHumor
group by post_date
order by post_date asc

post_date,count(1)
2022-06-17,23
2022-06-18,18
2022-06-19,79
2022-06-20,81
2022-06-21,57
2022-06-22,41
2022-06-23,64
2022-06-24,41
2022-06-25,56


In [10]:
%%sparksql

select * from reddit_ProgrammerHumor order by created_utc asc limit 10

approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,hidden,pwls,link_flair_css_class,downs,thumbnail_height,top_awarded_type,hide_score,name,quarantine,link_flair_text_color,upvote_ratio,author_flair_background_color,ups,total_awards_received,thumbnail_width,author_flair_template_id,is_original_content,secure_media,is_reddit_media_domain,is_meta,category,link_flair_text,can_mod_post,score,approved_by,is_created_from_ads_ui,author_premium,thumbnail,edited,author_flair_css_class,post_hint,content_categories,is_self,subreddit_type,created,link_flair_type,wls,removed_by_category,banned_by,author_flair_type,domain,allow_live_comments,selftext_html,likes,suggested_sort,banned_at_utc,url_overridden_by_dest,view_count,archived,no_follow,is_crosspostable,pinned,over_18,media_only,link_flair_template_id,can_gild,spoiler,locked,author_flair_text,visited,removed_by,mod_note,distinguished,subreddit_id,author_is_blocked,mod_reason_by,num_reports,removal_reason,link_flair_background_color,id,is_robot_indexable,report_reasons,author,discussion_type,num_comments,send_replies,whitelist_status,contest_mode,author_patreon_flair,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,post_date
null,ProgrammerHumor,,t2_54dvd6d4,False,null,0,False,"Never used python, ask me a question and I’ll answer like I did.",r/ProgrammerHumor,False,6,null,0,140,null,True,t3_ved7pz,False,dark,0.800000011920929,null,3,0,140,null,False,null,True,False,null,null,False,3,null,False,False,https://a.thumbs.redditmedia.com/J5kXwF3yIEX2tqDTiAZTmQr38_lKlI8I85PPooTfu20.jpg,False,null,image,null,False,public,2022-06-17 13:29:55.664896,text,6,null,null,text,i.redd.it,False,null,null,null,null,https://i.redd.it/i39ca2ufo6691.jpg,null,False,False,True,False,False,False,null,True,False,False,null,False,null,null,null,t5_2tex6,False,null,null,null,,ved7pz,True,null,TheBluePhenoix,null,5,False,all_ads,False,False,null,/r/ProgrammerHumor/comments/ved7pz/never_used_python_ask_me_a_question_and_ill/,all_ads,False,https://i.redd.it/i39ca2ufo6691.jpg,2056247,2022-06-17 13:29:55.664896,0,null,False,2022-06-17
null,ProgrammerHumor,,t2_75mqu1d5,False,null,0,False,The pain,r/ProgrammerHumor,False,6,meme,0,140,null,True,t3_vee1sy,False,dark,1.0,null,1,0,140,null,False,null,True,False,null,Meme,False,1,null,False,False,https://b.thumbs.redditmedia.com/vDJEMxOnIkDY7iJdkJ4v8rxt17S-3NGxeN4qO02kClw.jpg,False,null,image,null,False,public,2022-06-17 14:07:57.366272,richtext,6,null,null,text,i.redd.it,False,null,null,null,null,https://i.redd.it/qrbzub8fv6691.jpg,null,False,True,True,False,False,False,fe277d42-440f-11e8-8ec3-0eac434afc94,True,False,False,null,False,null,null,null,t5_2tex6,False,null,null,null,,vee1sy,True,null,mcalby,null,0,True,all_ads,False,False,null,/r/ProgrammerHumor/comments/vee1sy/the_pain/,all_ads,False,https://i.redd.it/qrbzub8fv6691.jpg,2056264,2022-06-17 14:07:57.366272,0,null,False,2022-06-17
null,ProgrammerHumor,,t2_5wtmwnwk,False,null,0,False,Running as an admin can make it run faster...,r/ProgrammerHumor,False,6,meme,0,119,null,True,t3_veeho2,False,dark,1.0,null,5,0,140,null,False,null,True,False,null,Meme,False,5,null,False,False,https://b.thumbs.redditmedia.com/LKEWm8rZ0uyom4SOTWKWSELv9RyqYqqvK7hwp7bBMSo.jpg,False,null,image,null,False,public,2022-06-17 14:30:19.543552,richtext,6,null,null,text,i.redd.it,False,null,null,null,null,https://i.redd.it/2bkimh0ow6691.png,null,False,False,True,False,False,False,fe277d42-440f-11e8-8ec3-0eac434afc94,True,False,False,null,False,null,null,null,t5_2tex6,False,null,null,null,,veeho2,True,null,vanessabaxton,null,1,True,all_ads,False,False,null,/r/ProgrammerHumor/comments/veeho2/running_as_an_admin_can_make_it_run_faster/,all_ads,False,https://i.redd.it/2bkimh0ow6691.png,2056353,2022-06-17 14:30:19.543552,0,null,False,2022-06-17
null,ProgrammerHumor,,t2_me0endh,False,null,0,False,This 

### stop spark

In [15]:
spark.stop()
print("stopped spark successfully.")

stopped spark successfully.
